# Corso di Probabilità e Statistica- Test statistici

In questo capitolo, faremo alcuni esempi di test statistici.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import scipy.stats 
import seaborn as sns
import matplotlib.pylab as plt

### Esempio: fratture 
Consideriamo di nuovo l'esempio della frattura della ceramica studiata nel capitolo 12

In [ ]:
#carichiamo il file
strength = pd.read_csv("../data/Strength_ceramic.dat",
                       sep="\s+",skiprows=48)
strength=strength.drop([0])

Ricordiamo le molte variabili presenti nel file. 
Vogliamo vedere quali di questi fattori hanno un impatto significativo sul carico di rottura Y

In [ ]:
with open("../data/Strength_ceramic.dat") as myfile:
    head = [next(myfile) for x in range(24)]
head


Controlliamo ad esempio il fattore X1 (Table speed).
Per questo separiamo il database in due set e confrontiamoli                     

In [ ]:
set1=strength[strength["X1"]==1.0].Y
set2=strength[strength["X1"]==-1.0].Y

Calcoliamo medie e varianze

In [ ]:
set1.mean(),set2.mean()

In [ ]:
set1.std(),set2.std()

Usiamo il t-test per testare l'ipotesi: I due set hanno la stessa media?

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

Dato un livello di significatività, che scegliamo ad esempio $\alpha=0.01$, se il test fornisce $p<\alpha$ possiamo rigettare l'ipotesi nulla che i due set abbiano la stessa media.

In [ ]:
scipy.stats.ttest_ind(set1,set2)

Interpretazione: $p > \alpha$ e quindi non possiamo rigettare l'ipotesi che i due set abbiano la stessa media. Il che non vuol dire che abbiamo dimostrato che la media sia la stessa...

Il T-test assume che le distribuzioni siano Gaussiane e con la stessa varianza. Controlliamo quindi gli istogrammi

In [ ]:
plt.hist(set1, bins=50, density=True, 
         histtype="step", label="Set 1" )
plt.hist(set2, bins=50, density=True, 
         histtype="step", label="Set 2")
plt.xlabel("X")
plt.ylabel("P(X)")
plt.legend()

Visto che la distribuzione non è Gaussiana utilizziamo invece  il test di Kolmogorov Smirnov.
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ks_2samp.html

Per prima cosa guardiamo le distribuzioni cumulate.

In [ ]:
xmin=np.min(strength.Y)
xmax=np.max(strength.Y)
x = np.linspace(xmin,xmax,101)
y1=np.sort(set1)
idx1 = [ i/len(y1) for i in range(len(y1))]
y2=np.sort(set2)
idx2 = [ i/len(y2) for i in range(len(y2))]
plt.xlabel('Y')
plt.ylabel('Cumulative Frequency')
plt.plot(y1,idx1,label="Set 1")
plt.plot(y2,idx2,label="Set 2")


In [ ]:
D,p_value=scipy.stats.ks_2samp(set1,set2)
D,p_value

Il test KS conferma il risultato del T-test. Il fattore X1 non ha un effetto significativo sul risultato.

**Esercizio** Eseguire il test di KS per tutti i possibili fattori e identificare i fattori significativi.

### Goodness of fit

Controlliamo se il fit con MLE è "buono" con il test di KS



In [ ]:
# Fittiamo con Weibull
k1, loc, x_1=scipy.stats.weibull_min.fit(set1, floc=0)
W1=scipy.stats.weibull_min(k1,loc=0, scale=x_1) 

In [ ]:
# Facciamo un grafico di confronto
xmin=np.min(strength.Y)
xmax=np.max(strength.Y)
x = np.linspace(xmin,xmax,101)
y1=np.sort(set1)
idx1 = [ i/len(y1) for i in range(len(y1))]
plt.xlabel('Y')
plt.ylabel('Cumulative Frequency')
plt.plot(y1,idx1,label="Set 1")
plt.plot(x, W1.cdf(x), label="Fit", color="black")
plt.legend()

Controlliamo ora il test:

In [ ]:
yy=set1.to_numpy()
scipy.stats.kstest(yy, 
                  lambda x: 
                   scipy.stats.weibull_min.cdf
                   (x, k1, loc=0, scale=x_1))

Il test ci da un valore $p>\alpha=0.01$ quindi non possiamo rifiutare l'ipotesi che i dati siano distribuiti secondo Weibull. 

Possiamo provare ora a ripetere l'esercizio supponendo invece che i dati siano Gaussiani

In [ ]:
mu_0=yy.mean()
sigma_0=yy.std()
mu_0,sigma_0

In [ ]:
scipy.stats.kstest(yy, 
                  lambda x: 
                   scipy.stats.norm.cdf
                   (x, loc=mu_0, scale=sigma_0))

In [ ]:
# Facciamo un grafico di confronto
xmin=np.min(strength.Y)
xmax=np.max(strength.Y)
x = np.linspace(xmin,xmax,101)
y1=np.sort(set1)
idx1 = [ i/len(y1) for i in range(len(y1))]
plt.xlabel('Y')
plt.ylabel('Cumulative Frequency')
plt.plot(y1,idx1,label="Set 1")
plt.plot(x, scipy.stats.norm.cdf
                   (x, loc=mu_0, scale=sigma_0), 
         label="Fit", color="red")
plt.legend()